In [1]:
import numpy as np
from pathlib import Path
import xarray as xr
import sarxarray

In [2]:
data = xr.open_dataset('stm.zarr')
data

/home/oku/miniforge3/envs/mobyle/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/oku/miniforge3/envs/mobyle/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/oku/miniforge3/envs/mobyle/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


<xarray.Dataset>
Dimensions:    (points: 318225, time: 10)
Coordinates:
    azimuth    (points) int64 ...
    lat        (points) float32 ...
    lon        (points) float32 ...
    range      (points) int64 ...
  * time       (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: points
Data variables:
    amplitude  (points, time) float32 ...
    complex    (points, time) complex64 ...
    phase      (points, time) float32 ...

## Float32

In [3]:
phase_float32 = data['phase'].astype(np.float32).compute()
phase_float32

<xarray.DataArray 'phase' (points: 318225, time: 10)>
array([[ 0.20038313,  0.99874884,  1.5353287 , ..., -2.1412406 ,
        -0.6583761 , -2.013114  ],
       [-2.323872  ,  0.507186  ,  2.5324106 , ...,  0.9834304 ,
         1.4191183 , -2.4606917 ],
       [ 2.7706344 , -0.45660147,  2.1859202 , ..., -0.17668577,
        -1.5304471 ,  1.5362003 ],
       ...,
       [-1.7213879 , -1.9891402 , -2.9865422 , ..., -0.6517241 ,
        -1.3779466 ,  2.387988  ],
       [-1.6791449 , -2.3804145 ,  2.895494  , ..., -1.4471537 ,
        -1.8687751 ,  1.624068  ],
       [-2.0037365 , -2.3618476 ,  2.8949456 , ..., -1.5446392 ,
        -1.8858798 ,  1.5394415 ]], dtype=float32)
Coordinates:
    azimuth  (points) int64 0 0 0 0 0 0 0 ... 1999 1999 1999 1999 1999 1999 1999
    lat      (points) float32 52.23 52.23 52.23 52.23 ... 52.5 52.5 52.5 52.5
    lon      (points) float32 4.81 4.816 4.818 4.818 ... 4.962 4.962 4.962 4.963
    range    (points) int64 31 124 165 168 169 172 ... 3990 3991 3992 3994 3995
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: points

In [4]:
# inspect memory usage of phase_float32, including coordinates
total_nbytes = phase_float32.nbytes # data
for coord in phase_float32.coords: # coordinates mem
    total_nbytes += phase_float32[coord].nbytes
print(f'{total_nbytes/1024/1024} MB')

19.422988891601562 MB


In [5]:
phase_float32.to_zarr('phase_float32.zarr', mode='w')
# inspect disk usage of phase_float32.zarr
!du -sh phase_float32.zarr

15M	phase_float32.zarr


In [6]:
# drop all coordinates
phase_float32_no_coords = phase_float32.reset_coords(drop=True)

# inspect memory usage of phase_float32
total_nbytes = phase_float32_no_coords.nbytes # data
for coord in phase_float32_no_coords.coords: # coordinates mem
    total_nbytes += phase_float32_no_coords[coord].nbytes
print(f'{total_nbytes/1024/1024} MB')
print(f'no coords float32: {total_nbytes/1024/1024} MB')

12.139396667480469 MB
no coords float32: 12.139396667480469 MB


In [7]:
phase_float32_no_coords.to_zarr('phase_float32_no_coords.zarr', mode='w')
!du -sh phase_float32_no_coords.zarr

13M	phase_float32_no_coords.zarr


## Float16

In [8]:
phase_float16 = data['phase'].astype(np.float16).compute()
phase_float16

<xarray.DataArray 'phase' (points: 318225, time: 10)>
array([[ 0.2004,  0.9985,  1.535 , ..., -2.14  , -0.658 , -2.014 ],
       [-2.324 ,  0.5073,  2.533 , ...,  0.9834,  1.419 , -2.46  ],
       [ 2.771 , -0.4565,  2.186 , ..., -0.1766, -1.53  ,  1.536 ],
       ...,
       [-1.722 , -1.989 , -2.986 , ..., -0.652 , -1.378 ,  2.389 ],
       [-1.679 , -2.38  ,  2.895 , ..., -1.447 , -1.869 ,  1.624 ],
       [-2.004 , -2.361 ,  2.895 , ..., -1.545 , -1.886 ,  1.539 ]],
      dtype=float16)
Coordinates:
    azimuth  (points) int64 0 0 0 0 0 0 0 ... 1999 1999 1999 1999 1999 1999 1999
    lat      (points) float32 52.23 52.23 52.23 52.23 ... 52.5 52.5 52.5 52.5
    lon      (points) float32 4.81 4.816 4.818 4.818 ... 4.962 4.962 4.962 4.963
    range    (points) int64 31 124 165 168 169 172 ... 3990 3991 3992 3994 3995
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: points

In [9]:
# inspect memory usage
total_nbytes = phase_float16.nbytes # data
for coord in phase_float16.coords: # coordinates mem
    total_nbytes += phase_float16[coord].nbytes
print(f'{total_nbytes/1024/1024} MB')
print(f'float16: {total_nbytes/1024/1024} MB')

13.353328704833984 MB
float16: 13.353328704833984 MB


In [10]:
phase_float16.to_zarr('phase_float16.zarr', mode='w')
!du -sh phase_float16.zarr

8.7M	phase_float16.zarr


In [11]:
# drop all coordinates
phase_float16_no_coords = phase_float16.reset_coords(drop=True)
phase_float16_no_coords
total_nbytes = phase_float16_no_coords.nbytes # data
for coord in phase_float16_no_coords.coords: # coordinates mem
    total_nbytes += phase_float16_no_coords[coord].nbytes
print(f'no coords float16: {total_nbytes/1024/1024} MB')

no coords float16: 6.069736480712891 MB


In [12]:
phase_float16_no_coords.to_zarr('phase_float16_no_coords.zarr', mode='w')
!du -sh phase_float16_no_coords.zarr

6.6M	phase_float16_no_coords.zarr
